## Estimating and apply multiple models in parrallel using Pandas UDFs and MLFlow
This use case invovles fitting multiple models in parallel to different groups of data. Each model is persisted in MLFlow. Then, we will apply the models to each group by loading each group's best model from MLFlow and performing a prediction. The dataset is based on the Titanic survival classification dataset. In this case 500 records were randomly chosen from that dataset and assigned to other well known shipwrecks. We will build a separate model in parallel for each ship wreck that estimates survival likelihood. This framework could easily be extended to very large datasets with many groups.

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

from pyspark.sql.types import StringType, DoubleType, StructType, StructField
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pyspark.sql.functions as func

import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Enable Arrow-based columnar data transfers
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [3]:
%sql

SELECT *
FROM default.workshop_many_models
LIMIT 5

name_prefix_Master,name_prefix_Miss,name_prefix_Mr,name_prefix_Mrs,name_prefix_None,name_parenths_no,name_parenths_yes,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Pclass_1,Pclass_2,Pclass_3,ticket_text_1,ticket_text_2,ticket_text_3,ticket_text_4,ticket_text_5,ticket_text_6,ticket_text_7,ticket_text_8,ticket_length_3,ticket_length_4,ticket_length_5,ticket_length_6,ticket_length_7,cabin_chars_A,cabin_chars_B,cabin_chars_C,cabin_chars_D,cabin_chars_E,cabin_chars_F,cabin_chars_INFREQ,cabin_chars_NONE,Age,Fare,SibSp,Parch,Survived,ship_name
0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29.69911764705882,110.8833,0.0,0.0,1.0,edmund_fitzgerald
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,45.0,35.5,0.0,0.0,0.0,edmund_fitzgerald
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,47.0,38.5,0.0,0.0,0.0,edmund_fitzgerald
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29.69911764705882,15.5,1.0,0.0,0.0,edmund_fitzgerald
0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,50.0,55.9,1.0,0.0,0.0,edmund_fitzgerald


In [4]:
%sql 

SELECT ship_name,
       count(*) as count
FROM default.workshop_many_models
GROUP BY ship_name

ship_name,count
arizona,500
edmund_fitzgerald,500
titanic,500
estonia,500


Notice that the table is partitioned by 'ship_name', the column we will group by when applying each model. This will improve perfomance on large datasets by mitigating the need to shuffle the data into groups before estimating and applying models

In [6]:
%sql

DESCRIBE TABLE EXTENDED default.workshop_many_models

col_name,data_type,comment
name_prefix_Master,double,null
name_prefix_Miss,double,null
name_prefix_Mr,double,null
name_prefix_Mrs,double,null
name_prefix_None,double,null
name_parenths_no,double,null
name_parenths_yes,double,null
Sex_female,double,null
Sex_male,double,null
Embarked_C,double,null


We can see the partition folder structure in within the file system

In [8]:
dbutils.fs.ls('dbfs:/user/hive/warehouse/workshop_many_models')

Out[34]: [FileInfo(path='dbfs:/user/hive/warehouse/workshop_many_models/_delta_log/', name='_delta_log/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/workshop_many_models/ship_name=arizona/', name='ship_name=arizona/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/workshop_many_models/ship_name=edmund_fitzgerald/', name='ship_name=edmund_fitzgerald/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/workshop_many_models/ship_name=estonia/', name='ship_name=estonia/', size=0),
 FileInfo(path='dbfs:/user/hive/warehouse/workshop_many_models/ship_name=titanic/', name='ship_name=titanic/', size=0)]

Set our Spark environment such that each task with have four cores to use. The idea is that each model fitting will have four cores available that sklearn can use to parrallelize its grid search.

In [10]:
spark.conf.set('spark.task.cpus', 4)
spark.conf.set('spark.executor.cores', 4)

print(spark.conf.get('spark.task.cpus'), spark.conf.get('spark.executor.cores'))

4 4

### Define the Pandas UDF
Define a UDF that will fit an sklearn model to a group of data; the models will be stored in MLFlow, and the model scores will be returned for each group.

In [12]:
def fit_models_config(schema, features_cols, label_col, grouping_col, classifier, param_grid, scoring, experiment_location, cv=5):
  
  """Apply a scikit learn model to a group of data within a Spark DataFrame using a Pandas UDF
  
  Arguments:
  schema: Spark DataFrame schema: A Spark DataFrame schema that maps to the output of the function
  features_cols: List[str]:       List of column names that represent the model features
  label_col: str:                 Column to be predicted
  grouping_col: str:                 The column on which the DataFrame is being grouped
  classier: sklearn classifier:   Classifier to use
  param_grid: Dict                Grid search data structure containing the parameters to search
  scoring: str                    Scoring method to use for validation
  experiment_location: str        Path to the MLFlow experiment
  cv: int                         Number of cross validation folds

  """

  @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
  def fit_models(data):
    """Fit the model; log the best model and its paramenters to 
    MLFlow"""

    # Specify features and label
    features = data[features_cols]
    label = data[label_col]

    # Create and fit model
    clf = classifier
    grid_search = GridSearchCV(clf, param_grid, scoring=scoring, n_jobs=-1, cv=cv)
    grid_search.fit(features, label)

    # Log metrics and artifacts to MLFlow
    mlflow.set_experiment(experiment_location)

    with mlflow.start_run() as run:

      group_name = data[grouping_col].loc[0]

      best_model_score = grid_search.best_score_

      model_results_df = pd.DataFrame([(group_name, best_model_score)], 
                                        columns= ['group_name', 'best_model_score'])

      mlflow.set_tag("group_name", group_name)

      mlflow.set_tag("classifier_type", type(clf).__name__)

      mlflow.log_metric(scoring, best_model_score)
      
      best_params = grid_search.best_params_
      
      mlflow.log_params(best_params)

      mlflow.sklearn.log_model(sk_model=grid_search.best_estimator_, 
                               artifact_path='survival_model')

      return model_results_df
    
  return fit_models

Specify the features DataFrame, label column name and features column names

In [14]:
spark_features = spark.table('default.workshop_many_models')

label_col = 'Survived'
features_cols = [column for column in spark_features.columns if column not in [label, 'ship_name']]

Configure the Pandas UDF

In [16]:
# Pandas_UDF requires a Spark Schema that matches the output of the UDF
fit_schema = StructType([StructField('group_name', StringType(), True),
                         StructField('best_model_score', DoubleType(), True)])

fit_models = fit_models_config(schema =        fit_schema, 
                               features_cols = features_cols, 
                               label_col =     label_col,
                               grouping_col =  "ship_name",
                               classifier =    RandomForestClassifier(n_jobs=-1),
                               param_grid =    {'n_estimators':       [20, 50, 100], 
                                                 'min_samples_split': [2, 5, 10],
                                                 'max_features':      [2, 5, 10]},
                                scoring =       "roc_auc",
                                experiment_location = "/Users/marshall.carter@databricks.com/workshops/ml/pandas_udf/survival_prediction"
                                )

### Fit and store a model for each ship

In [18]:
best_model_stats = spark_features.groupBy('ship_name').apply(fit_models)

display(best_model_stats)

group_name,best_model_score
arizona,0.8531899665365668
edmund_fitzgerald,0.8598849504471425
titanic,0.8519684377522335
estonia,0.8543395821245747


A View of the models populated in MLFlow by our Pandas_UDF. At this point we have estimated only one model per group, but we could easily fit many models. Note: the below image is from an older model run.

<img src="files/marshall.carter/workshop_many_models.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

### Create a Pandas UDF to apply the models  
The UDF will find the relevent model for each group within MLFlow and preform a prediction for that group. In this example, the best model for each group is being chosen by the 'score' metric. A more production focused method for applying the models would be to register each group's best model in the Model Registry and then load the model's from the registry. See the Model registry [documentation](https://docs.databricks.com/applications/mlflow/model-registry.html) and [example notebook](https://docs.databricks.com/_static/notebooks/mlflow/mlflow-model-registry-example.html).

In [21]:
def apply_models_config(schema, features_cols, grouping_col, score="roc_auc", experiment_id="3007990057292469"):
  
  """For each distinct group (values in groupBy statement), load the group's best model and 
  perform a prediction
  
  Arguments:
  schema: Spark DataFrame schema: A Spark DataFrame schema that maps to the output of the function
  features_cols: List[str]:       List of column names that represent the model features
  grouping_col: str:              The column on which the DataFrame is being grouped
  scoring: str                    Scoring method to use for selecting the best model
  experiment_id: str              The id of the experiment from which to select models
  
  """
  
  @pandas_udf(schema, PandasUDFType.GROUPED_MAP)
  def apply_models(data):
    """Load the relvent model for the selected group and generate a
    prediciton for the group"""
  
    group_name = data[grouping_col].loc[0]
    
    
    def get_model_info(run_id):
      """Get the ship name, run_id, and scoring metric of each run in 
      the experiment"""

      data = client.get_run(run_id).data

      fitted_model_group_name = data.tags['group_name']
      metric = data.metrics[score]

      return (fitted_model_group_name, run_id, metric)


    # Get all of the runs in the MLFlow experiment
    runs = client.list_run_infos(experiment_id)

    # Get the run_ids for each run
    run_ids = [run.run_id for run in runs]

    # Get all relevent infor for each model run
    models = [get_model_info(run_id) for run_id in run_ids]

    # Filter to only models built using this group's data
    models_for_group = [model for model in models if model[0] == group_name]

    # Find the best model for this group by sorting descending by the scoring metric
    best_model_for_group = sorted(models_for_group, key = lambda x: x[2], reverse=True)[0]
  
    best_model_run_id = best_model_for_group[1]

    # Load the best model via its run_id
    loaded_model = mlflow.sklearn.load_model(f"runs:/{best_model_run_id}/survival_model")

    # Perform prediction; combine features and predictions
    predictions = loaded_model.predict(data[features_cols])
    predictions_df = pd.DataFrame(predictions, columns=['prediction'])

    features_and_predictions = pd.concat([predictions_df, data], axis=1)
    features_and_predictions['run_id'] = best_model_run_id

    return features_and_predictions

  return apply_models

Configure the Pandas UDF

In [23]:
prediction_schema = StructType()
prediction_schema.add('prediction', DoubleType())
prediction_schema.add('run_id', StringType())

for column in spark_features.schema:
  prediction_schema.add(column.name, column.dataType)
  

apply_models = apply_models_config(schema =        prediction_schema, 
                                   features_cols = features_cols,
                                   grouping_col =  "ship_name"
                                    )

#### Apply the models to each ship

In [25]:
predictions = spark_features.groupBy('ship_name').apply(apply_models)

display(predictions)

prediction,run_id,name_prefix_Master,name_prefix_Miss,name_prefix_Mr,name_prefix_Mrs,name_prefix_None,name_parenths_no,name_parenths_yes,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan,Pclass_1,Pclass_2,Pclass_3,ticket_text_1,ticket_text_2,ticket_text_3,ticket_text_4,ticket_text_5,ticket_text_6,ticket_text_7,ticket_text_8,ticket_length_3,ticket_length_4,ticket_length_5,ticket_length_6,ticket_length_7,cabin_chars_A,cabin_chars_B,cabin_chars_C,cabin_chars_D,cabin_chars_E,cabin_chars_F,cabin_chars_INFREQ,cabin_chars_NONE,Age,Fare,SibSp,Parch,Survived,ship_name
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29.69911764705882,8.05,0.0,0.0,0.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29.69911764705882,14.4583,0.0,0.0,0.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,19.0,8.05,0.0,0.0,1.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,28.0,7.8542,0.0,0.0,0.0,arizona
1.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,45.0,26.55,0.0,0.0,1.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,24.0,7.05,0.0,0.0,0.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,29.69911764705882,8.4583,0.0,0.0,0.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,10.0,24.15,0.0,2.0,0.0,arizona
1.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,58.0,153.4625,0.0,1.0,1.0,arizona
0.0,c441321fa1034754997c2e74b38793b5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,44.0,26.0,1.0,0.0,0.0,arizona


Confirm that different models were used for each ship

In [27]:
display(
  predictions.groupBy(['ship_name', 'run_id']).agg(func.count("*").alias("count"))
)

ship_name,run_id,count
arizona,c441321fa1034754997c2e74b38793b5,500
titanic,5433df0b91024b3b8e9f93c0033ae736,500
edmund_fitzgerald,3097ca31e1d84534992ff96fcd64044d,500
estonia,3a4e66eee0f6420faf8716a418c024fe,500


View the prediction for each ship

In [29]:
display(
  predictions.groupBy('ship_name').agg(func.sum("prediction").alias("survived"))
)

ship_name,survived
arizona,169.0
edmund_fitzgerald,167.0
titanic,163.0
estonia,169.0
